In [7]:
import csv
import numpy as np 
from scipy import stats
import igraph

In [8]:
list_jan_data = [] 
list_feb_data = [] 
list_mar_data = [] 
list_all_data = [] 
with open('Uber_Data/los_angeles-censustracts-2020-1-All-MonthlyAggregate.csv', 'r') as f1:
    for line in f1:
        line =line.strip('\n')
        arr = line.split(',')
        #print(arr[2])
        list_all_data.append(arr) 
        if arr[2] == '01':
            list_jan_data.append(arr) 
        if arr[2] == '02':
            list_feb_data.append(arr)
        if arr[2] == '03':
            list_mar_data.append(arr) 
            
        

In [9]:
# Make everything into a numpy array, rows are still for each entry and cols are attributes 
list_jan_data = np.array(list_jan_data)
list_feb_data = np.array(list_feb_data)
list_mar_data = np.array(list_mar_data)
list_all_data = np.array(list_all_data)


In [10]:
print(list_mar_data[:,0]) 

['1575' '939' '988' ... '167' '585' '679']


In [6]:
# Find the number that appears the most as the source or destination. 
# This will be the placeholder for DTLA until we find it
sources = list_all_data[:,0] 
dests = list_all_data[:,1] 
all_locs = np.vstack((sources,dests)) 
m = stats.mode(all_locs)[0]
print(m)


[['dstid' '1575' '125' ... '1236' '2596' '1656']]


In [11]:
# "Source" is the "main node" the placeholder for DTLA 
source = '1575'
# Iterate through the list 
# I tried this with January + February but it made less sense 
#pre_pandemic = np.vstack((list_jan_data,list_feb_data)) 
pre_pandemic = list_jan_data

post_pandemic = list_mar_data 

# Find the pre-pandemic entries from location '1575' that take less than 30 mins 
# Initialize np array with 0 rows and correct # of cols 
pre_pandemic_less_30 = np.zeros((0,pre_pandemic.shape[1]))

# Iterate row by row 
for line_num in range(pre_pandemic.shape[0]):
    # Check source location 
    if pre_pandemic[line_num,0] == source:
        # Check that mean travel time is less than 30 mins (or 1800 seconds) 
        if float(pre_pandemic[line_num,3]) < 1800.0:
            pre_pandemic_less_30 = np.vstack((pre_pandemic_less_30,pre_pandemic[line_num,:]))
            
    
# Find the post-pandemic entries from location '1575' that take less than 30 mins 
# Initialize np array with 0 rows and correct # of cols 
post_pandemic_less_30 = np.zeros((0,post_pandemic.shape[1]))

# Iterate row by row 
for line_num in range(post_pandemic.shape[0]):
    # Check source location 
    if post_pandemic[line_num,0] == source:
        # Check that mean travel time is less than 30 mins (or 1800 seconds) 
        if float(post_pandemic[line_num,3]) < 1800.0:
            post_pandemic_less_30 = np.vstack((post_pandemic_less_30,post_pandemic[line_num,:]))    


In [12]:
print(pre_pandemic.shape)
print(pre_pandemic_less_30.shape)
print(post_pandemic.shape)
print(post_pandemic_less_30.shape)

(1564592, 7)
(1072, 7)
(1247442, 7)
(1114, 7)


Despite the fact that there are originally more entries in January than in March, there are more places you can get to in 30 min from location 1575 in March than in January. 

In [13]:
print(pre_pandemic_less_30)

[['1575' '1430' '01' ... '629.62' '1357.72' '1.44']
 ['1575' '1383' '01' ... '647.27' '971.08' '1.64']
 ['1575' '1026' '01' ... '481.42' '625.24' '1.61']
 ...
 ['1575' '1177' '01' ... '728.57' '1344.92' '1.49']
 ['1575' '1343' '01' ... '350.7' '441.44' '1.69']
 ['1575' '984' '01' ... '648.11' '1384.73' '1.42']]


In [26]:
# Make two graphs: One for pre-pandemic and one post-pandemic  
# The nodes: '1575' and every number that exists in the second column above (destinations) 
# The edges: An edge exists between two nodes if their travel time is less than 30 mins 
# So an edge can exist between two nodes even if neither is '1575' 

# Number of vertices pre-pandemic 
num_vertices_pre = 1+pre_pandemic_less_30.shape[0] 
# Create a directed graph
g_pre = igraph.Graph(directed=True)
# Add num_vertices_pre vertices
g_pre.add_vertices(num_vertices_pre)

# Add ids and labels to vertices
# First label is '1575' and then iterate through the rest 
g_pre.vs[0]["id"] = 0 
g_pre.vs[0]["label"] = '1575'  
for i in range(1,len(g_pre.vs)):
    #print(i)
    g_pre.vs[i]["id"]= i
    # Label is the location id 
    g_pre.vs[i]["label"]= pre_pandemic_less_30[i-1,1] 


node_labels = g_pre.vs["label"]


In [ ]:
# For all the node labels 
# Iterate through each row of list_jan_data 
# if source and dest are both in node_labels
# if the travel time is less than 30 mins 
# If an edge doesn't already exist between these two nodes
# Add an edge between these two nodes 
# And add this line to a new matrix in case we need it for later 
